In [1]:
import os
os.chdir(os.path.join(os.getcwd(), '..'))

In [2]:
import json
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
from keras.models import load_model
from src.custom_loss import weighted_sum_loss
from src.model import mean_teacher
from src.data_generator import mean_teacher_data_gen
import numpy as np
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [3]:
config = K.tf.ConfigProto()
config.gpu_options.allow_growth = True
session = K.tf.InteractiveSession(config=config)  

In [4]:
WEIGHTS_PATH = os.path.join(os.getcwd(), 'weights', 'model_1-894-0.00.hdf5')
REDUCED_DATA_PATH = os.path.join(os.getcwd(), 'reduced_data')

In [5]:
with open(os.path.join(os.getcwd(), 'configs', 'model.json')) as f:
    model_configs = json.load(f)
    
with open(os.path.join(os.getcwd(), 'configs', 'training.json')) as f:
    training_configs = json.load(f)

In [31]:
real_data_gen = ImageDataGenerator().flow_from_directory(os.path.join(REDUCED_DATA_PATH, 'real'),
                                                        target_size=training_configs['target_size'], seed = 99)
syn_data_gen = ImageDataGenerator().flow_from_directory(os.path.join(REDUCED_DATA_PATH, 'synthetic'),
                                                     target_size=training_configs['target_size'])

Found 1200 images belonging to 12 classes.
Found 1200 images belonging to 12 classes.


In [32]:
data_gen = mean_teacher_data_gen(ImageDataGenerator(), ImageDataGenerator(), training_configs['batch_size'], training_configs['target_size'])

In [8]:
mean_teacher_model, student_model, teacher_model = mean_teacher(model_configs)

Instructions for updating:
Colocations handled automatically by placer.


/home/dh/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [10]:
mean_teacher_model.load_weights(WEIGHTS_PATH)

In [33]:
steps_needed = real_data_gen.n//real_data_gen.batch_size + 1
real_data_gen.reset()
pred = mean_teacher_model.predict_generator(data_gen, steps = steps_needed, verbose = 1)

Found 1200 images belonging to 12 classes.
Found 1200 images belonging to 12 classes.
38/38 [==============================] - 6s 148ms/step


In [34]:
predicted_class_indices=np.argmax(pred,axis=1)

In [35]:
labels = (real_data_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())
class_labels = list(labels.values())
predictions = [labels[k] for k in predicted_class_indices]

In [41]:
data_gen = mean_teacher_data_gen(ImageDataGenerator(), ImageDataGenerator(), training_configs['batch_size'], training_configs['target_size'])

In [42]:
mean_teacher_model.evaluate_generator(data_gen, steps = steps_needed, verbose = 1)

Found 1200 images belonging to 12 classes.
Found 1200 images belonging to 12 classes.
38/38 [==============================] - 16s 416ms/step


[26.90649408976237, 1.0]

In [36]:
real_data_gen.reset()
Y_test = []
while True:
    x1,y1 = real_data_gen.next()
    x2,y2 = next(data_gen)
    print(np.array_equal(y1,y2))

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


KeyboardInterrupt: 

In [ ]:
confusion = confusion_matrix(Y_test, predictions)
NUM_LABELS = real_data_gen.n
f, axes = plt.subplots(1,1, figsize = (12,12))
axes.set_xlabel('Actual')
axes.set_ylabel('Predicted')
axes.grid(False)
axes.set_xticklabels(class_labels, rotation = 90)
axes.set_yticklabels(class_labels)
axes.set_yticks(list(range(NUM_LABELS)))
axes.set_xticks(list(range(NUM_LABELS)))
plt.imshow(confusion, cmap=plt.cm.Set2, interpolation='nearest')

for i, cas in enumerate(confusion):
    for j, count in enumerate(cas):
        if count > 0:
            xoff = .07 * len(str(count))
            plt.text(j-xoff, i+.2, int(count), fontsize=12, color='black')